In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathos.pools import ProcessPool as Pool

In [ ]:
from boxas.config import get_config
from boxas.runners import runner
from boxas.loss import LossFn
from boxas.xasopt import XASOpt
from boxas.utils import header_template
from boxas.exp_data import load_spectrum

/home/ra1513/work/codes/bo_xas_github/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
cfg = get_config('cfg_foils.yaml')
project_root = cfg['dir']['project_root']

In [5]:
opt_params = ['Vi', 'r_scf', 'r_fms']
context = {k:v for k,v in cfg['feff_params'].items() if k in opt_params}
metrics = ['l2_norm_dist']#, 'r_factor', 'cosine_sim', 'pearson_corr', 'spearman_corr']

template = header_template(cfg, code='feff', region='xanes')

def optimize_xas(N, material, compounds, metric):
    loss_fn = LossFn(N, metric=metric)

    opt = XASOpt(
        f"feff_{material}_{metric}",
        context, 
        template, 
        compounds,
        runner, 
        loss_fn
    )

    best_params = opt.optimize(
        n_trials=10,            # increase the number of trials as needed
        patience=10,
        load_if_exists=True     # Set to True if you want to resume a previous study
    )

    return best_params


In [6]:
for m in ['Ni_aps']:#, 'Fe_aps', 'Pd_bioxas']:

    e0 = cfg['exp_data'][m]['energy']['edge']
    compounds = cfg['exp_data'][m]['compounds']
    N = load_spectrum(
        cfg['exp_data'][m], 
        m
    )
    
    pool = Pool(nodes=6)
    results = pool.map(
        lambda met: optimize_xas(N, m, compounds, met),
        metrics
    )

    pool.clear()

Ni_aps :  8133.0 9233


[I 2025-10-24 11:43:33,459] A new study created in RDB with name: feff_Ni_aps_l2_norm_dist
[I 2025-10-24 11:43:54,404] Trial 0 finished with value: 0.0740375690656832 and parameters: {'Vi': 0.8780179853398823, 'r_scf': 4.181571189951193, 'r_fms': 4.168656493975135}. Best is trial 0 with value: 0.0740375690656832.
[I 2025-10-24 11:44:56,270] Trial 1 finished with value: 0.07346423819592691 and parameters: {'Vi': 1.8205168315527969, 'r_scf': 4.456640794404011, 'r_fms': 4.122200068562916}. Best is trial 1 with value: 0.07346423819592691.
[I 2025-10-24 11:47:01,810] Trial 2 finished with value: 0.03720387386268288 and parameters: {'Vi': 1.2178473940808114, 'r_scf': 4.995359663593065, 'r_fms': 4.849356167896057}. Best is trial 2 with value: 0.03720387386268288.
[I 2025-10-24 11:48:02,627] Trial 3 finished with value: 0.07558983579191228 and parameters: {'Vi': 0.7332633035083987, 'r_scf': 4.712739701178788, 'r_fms': 4.035487132518963}. Best is trial 2 with value: 0.03720387386268288.
[I 2025

Best trial of feff_Ni_aps_l2_norm_dist: 8
Best value: 0.03699213154997036
Best parameters: {'Vi': 0.6504227614974019, 'r_scf': 4.80083598049631, 'r_fms': 4.713345401916119}
